In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# Load the data
excel_file_path = '/content/ENB2012_data.xlsx'

# Loading the data from the Excel file
data = pd.read_excel(excel_file_path)



FileNotFoundError: [Errno 2] No such file or directory: '/content/ENB2012_data.xlsx'

X1	Relative Compactness

X2	Surface Area

X3	Wall Area

X4	Roof Area

X5	Overall Height

X6	Orientation

X7	Glazing Area

X8	Glazing Area Distribution

y1	Heating Load

y2	Cooling Load

In [ ]:
# 1. Data Preprocessing

# Splitting the data into features (X) and target variables (y1 and y2)
X = data.drop(columns=['Y1', 'Y2'])
y1 = data['Y1']
y2 = data['Y2']

# Splitting the data into training and testing sets for both target variables
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y1, test_size=0.3, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size=0.3, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)
X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)


In [ ]:
# Defining the models
models = [
    ('Linear Regression', LinearRegression()),
    ('Decision Tree', DecisionTreeRegressor(random_state=42)),
    ('Random Forest', RandomForestRegressor(random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(random_state=42)),
    ('Support Vector Regression', SVR())
]

# Placeholder for results for Heating Load and Cooling Load
results_heating = []
results_cooling = []

# Iterate over models, train, predict, and evaluate for Heating Load
for name, model in models:
    model_clone = model
    model_clone.fit(X_train1, y_train1)  # Training
    y_pred = model_clone.predict(X_test1)  # Predictions
    r2 = r2_score(y_test1, y_pred)  # Evaluation using R-squared
    results_heating.append((name, r2))

# Iterate over models, train, predict, and evaluate for Cooling Load
for name, model in models:
    model_clone = model
    model_clone.fit(X_train2, y_train2)  # Training
    y_pred = model_clone.predict(X_test2)  # Predictions
    r2 = r2_score(y_test2, y_pred)  # Evaluation using R-squared
    results_cooling.append((name, r2))

# Sort models based on R-squared (higher is better)
sorted_results_heating = sorted(results_heating, key=lambda x: x[1], reverse=True)
sorted_results_cooling = sorted(results_cooling, key=lambda x: x[1], reverse=True)

sorted_results_heating, sorted_results_cooling

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import joblib

# Train Random Forest for Heating Load
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train1, y_train1)

# Train Gradient Boosting for Cooling Load
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train2, y_train2)

# Save the models
joblib.dump(rf_model, 'random_forest_heating_model.pkl')
joblib.dump(gb_model, 'gradient_boosting_cooling_model.pkl')


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import joblib

# Train Random Forest for Heating Load
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train1, y_train1)

# Train Gradient Boosting for Cooling Load
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train2, y_train2)

# Save the models
joblib.dump(rf_model, 'random_forest_heating_model.pkl')
joblib.dump(gb_model, 'gradient_boosting_cooling_model.pkl')


In [ ]:
from flask import Flask, request, render_template
import joblib

app = Flask(__name__)

# Load the models
rf_model = joblib.load('random_forest_heating_model.pkl')
gb_model = joblib.load('gradient_boosting_cooling_model.pkl')

@app.route('/')
def home():
    return render_template('index.html')
@app.route('/predict', methods=['POST'])
def predict():
    input_features = [float(x) for x in request.form.values()]
    heating_load = rf_model.predict([input_features])[0]
    cooling_load = gb_model.predict([input_features])[0]
    return render_template('index.html', heating_prediction=heating_load, cooling_prediction=cooling_load)

if __name__ == '__main__':
    app.run(debug=True)
